In [1]:
import h2o
h2o.init()
from h2o.estimators.word2vec import H2OWord2vecEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,6 days 20 hours 11 mins
H2O cluster timezone:,Europe/Vienna
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.5
H2O cluster version age:,"14 days, 11 hours and 31 minutes"
H2O cluster name:,H2O_from_python_mackenzie_dm3nzl
H2O cluster total nodes:,1
H2O cluster free memory:,1.375 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [2]:
job_titles_path = "https://raw.githubusercontent.com/h2oai/sparkling-water/rel-1.6/examples/smalldata/craigslistJobTitles.csv"
job_titles = h2o.import_file(job_titles_path, destination_frame = "jobtitles",
                             col_names = ["category", "jobtitle"], col_types = ["enum", "string"], header = 1)
print(job_titles)

Parse progress: |█████████████████████████████████████████████████████████| 100%


category,jobtitle
education,After School Supervisor
education,"*****TUTORS NEEDED - FOR ALL SUBJECTS, ALL AGES*****"
education,Bay Area Family Recruiter
education,Adult Day Programs/Community Access/Job Coaches
education,General Counselor - Non Tenure track
education,Part-Time Summer Math Teachers/Tutors
education,Preschool Teacher (temp-to-hire)
education,"*****TUTORS NEEDED - FOR ALL SUBJECTS, ALL AGES*****"
education,Private Teachers and Tutors Needed in the South Bay
education,Art Therapist at Esther B. Clark School


In [3]:
STOP_WORDS = ["ax","i","you","edu","s","t","m","subject","can","lines","re","what",
               "there","all","we","one","the","a","an","of","or","in","for","by","on",
               "but","is","in","a","not","with","as","was","if","they","are","this","and","it","have",
               "from","at","my","be","by","not","that","to","from","com","org","like","likes","so"]

In [4]:
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [5]:
def predict(job_title,w2v, gbm):
    words = tokenize(h2o.H2OFrame(job_title).ascharacter())
    job_title_vec = w2v.transform(words, aggregate_method="AVERAGE")
    print(gbm.predict(test_data=job_title_vec))

In [6]:
print("Break job titles into sequence of words")
words = tokenize(job_titles["jobtitle"])

Break job titles into sequence of words


In [7]:
print("Build word2vec model")
w2v_model = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 10)
w2v_model.train(training_frame=words)

Build word2vec model
word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [8]:
print("Sanity check - find synonyms for the word 'teacher'")
w2v_model.find_synonyms("teacher", count = 5)

Sanity check - find synonyms for the word 'teacher'


OrderedDict([('preschool', 0.6674632430076599),
             ('infant', 0.666761577129364),
             ('teaching', 0.6467991471290588),
             ('toddler', 0.6421365141868591),
             ('aide', 0.637873649597168)])

In [14]:
print("Calculate a vector for each job title")
job_title_vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")
print(job_title_vecs)

Calculate a vector for each job title


C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
-0.251584,0.136924,-0.255622,0.312074,-0.0118113,-0.0419435,0.0466443,-0.22886,-0.303602,-0.0472739,-0.0213124,-0.0703419,0.0531935,-0.408683,-0.0789673,-0.094132,-0.13789,-0.163174,-0.3242,0.276175,0.304799,-0.309087,-0.131799,-0.187872,0.371162,-0.158022,0.101008,-0.062833,0.300701,-0.372577,0.041656,-0.233302,-0.0720808,-0.320965,0.0344183,-0.0503962,0.0385672,0.137535,0.10059,-0.0508965,-0.105906,0.0940593,0.335523,0.0797369,0.124125,0.0151864,0.196927,0.214675,-0.263531,-0.135715,-0.178507,0.107794,-0.162345,-0.12102,-0.158771,-0.127631,0.42478,-0.00291389,0.566371,-0.266624,0.124889,-0.0726887,-0.0131716,-0.102916,0.139192,0.183915,0.116067,0.0347701,-0.0838,0.0197136,0.169457,-0.0313154,0.00486292,-0.106337,-0.185604,-0.457041,-0.0265876,0.202438,-0.268457,-0.346918,0.38533,0.00512209,0.253923,0.413862,0.0142401,0.172131,-0.36137,0.114691,-0.470486,-0.0347269,-0.0808012,-0.160032,0.0217447,-0.143188,0.0275136,-0.147386,-0.41983,-0.133007,0.112142,-0.420146
-0.385594,-0.223537,-0.589687,-0.214223,-0.362832,-0.430404,0.537216,0.737671,-0.118072,0.194709,0.184864,-0.0461948,0.307238,-0.153701,-0.0568609,-0.195524,-0.449524,0.00160749,-0.291577,0.295948,0.545097,0.590229,-0.332481,0.0696192,-0.415839,-0.27066,-0.574071,-0.167856,-0.130402,0.682819,-0.0527046,0.275407,-0.453048,-0.0699872,0.0170623,0.365132,-0.386299,0.160154,-0.0564246,-0.107434,-0.413609,0.300056,0.850056,-0.04601,0.222707,-0.0114921,0.164838,0.415203,-0.294713,0.187536,-0.0324484,0.0921537,-0.23483,-0.152381,0.312211,0.307891,0.011294,-0.0851305,0.300332,-0.0556199,0.342629,0.0971193,0.142128,-0.650731,0.296569,-0.0170683,0.00161196,0.101912,-0.214938,-0.243685,0.0224212,0.266922,-0.00427749,0.000947144,-0.155636,0.373267,0.317644,0.0355001,0.265651,0.276324,-0.0219808,0.0672442,0.200542,0.471807,0.0344986,-0.0202369,-0.303343,0.335097,0.0966534,-0.313922,-0.337812,-0.0620166,-0.185626,0.0933087,0.0755286,-0.0661411,-0.815489,-0.170913,0.235029,-0.180461
-0.0954117,-0.0702975,-0.186445,-0.00935915,0.0431085,0.090021,0.119256,0.0611851,-0.295891,0.231734,-0.170702,0.13069,0.277911,0.222596,-0.302772,0.0669213,0.158227,0.118516,0.0563594,0.154408,0.0242654,0.308068,0.0789735,0.0111396,0.00101944,-0.0425005,0.110333,0.080065,0.255814,-0.0254293,0.36138,0.276812,-0.0964162,0.129942,0.130713,-0.184794,0.0156939,-0.0654218,0.0128636,0.47527,-0.226724,0.314848,-0.0730507,-0.216765,-0.0707427,-0.00603081,0.0699789,-0.14366,-0.364797,-0.0559318,-0.129343,0.197252,-0.107991,-0.11598,0.249636,0.127396,0.0745447,-0.207772,0.0444609,-0.092851,0.116533,-0.00733192,0.208556,-0.187216,0.120188,0.0866311,0.244091,0.146689,0.111703,-0.334957,0.17488,-0.0725898,-0.223959,-0.012875,0.229429,-0.140747,0.0467826,-0.20798,-0.0667032,0.207997,-0.106177,-0.101408,0.0615601,0.151133,-0.254149,0.15218,-0.208574,0.152721,-0.188204,-0.0652586,-0.0742955,-0.019335,0.0973487,0.0386987,0.196887,-0.0808491,-0.013056,-0.107551,-0.0197343,-0.0897458
-0.0633724,0.0697162,0.045874,-0.132308,-0.0125692,-0.0697526,0.349191,0.0143475,-0.14405,0.0573575,0.0704745,-0.0583423,0.134514,-0.171774,-0.268729,-0.192195,-0.174414,-0.150645,-0.19068,0.0681866,0.024791,-0.0256565,-0.0828449,-0.116476,0.224361,-0.0730649,-0.283766,-0.201799,0.165288,-0.30559,0.0220558,0.117959,-0.125571,-0.084034,-0.0111401,0.0747539,0.101458,0.0188188,-0.0222562,0.0422242,-0.0116395,0.0998284,-0.00727211,0.000483806,-0.000156656,-0.207639,0.0446618,0.266029,-0.24315,-0.0656087,-0.34511,0.207761,0.0320722,-0.0974651,-0.0964021,-0.014266,0.228001,0.0399858,0.1847,-0.0735165,0.00450312,0.331971,0.0470354,0.0704103,0.117919,-0.189733,0.38288

In [15]:
print("Prepare training&validation data (keep only job titles made of known words)")
valid_job_titles = ~ job_title_vecs["C1"].isna()
data = job_titles[valid_job_titles,:].cbind(job_title_vecs[valid_job_titles,:])
data_split = data.split_frame(ratios=[0.8])
print(data)

Prepare training&validation data (keep only job titles made of known words)


category,jobtitle,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
education,After School Supervisor,-0.251584,0.136924,-0.255622,0.312074,-0.0118113,-0.0419435,0.0466443,-0.22886,-0.303602,-0.0472739,-0.0213124,-0.0703419,0.0531935,-0.408683,-0.0789673,-0.094132,-0.13789,-0.163174,-0.3242,0.276175,0.304799,-0.309087,-0.131799,-0.187872,0.371162,-0.158022,0.101008,-0.062833,0.300701,-0.372577,0.041656,-0.233302,-0.0720808,-0.320965,0.0344183,-0.0503962,0.0385672,0.137535,0.10059,-0.0508965,-0.105906,0.0940593,0.335523,0.0797369,0.124125,0.0151864,0.196927,0.214675,-0.263531,-0.135715,-0.178507,0.107794,-0.162345,-0.12102,-0.158771,-0.127631,0.42478,-0.00291389,0.566371,-0.266624,0.124889,-0.0726887,-0.0131716,-0.102916,0.139192,0.183915,0.116067,0.0347701,-0.0838,0.0197136,0.169457,-0.0313154,0.00486292,-0.106337,-0.185604,-0.457041,-0.0265876,0.202438,-0.268457,-0.346918,0.38533,0.00512209,0.253923,0.413862,0.0142401,0.172131,-0.36137,0.114691,-0.470486,-0.0347269,-0.0808012,-0.160032,0.0217447,-0.143188,0.0275136,-0.147386,-0.41983,-0.133007,0.112142,-0.420146
education,"*****TUTORS NEEDED - FOR ALL SUBJECTS, ALL AGES*****",-0.385594,-0.223537,-0.589687,-0.214223,-0.362832,-0.430404,0.537216,0.737671,-0.118072,0.194709,0.184864,-0.0461948,0.307238,-0.153701,-0.0568609,-0.195524,-0.449524,0.00160749,-0.291577,0.295948,0.545097,0.590229,-0.332481,0.0696192,-0.415839,-0.27066,-0.574071,-0.167856,-0.130402,0.682819,-0.0527046,0.275407,-0.453048,-0.0699872,0.0170623,0.365132,-0.386299,0.160154,-0.0564246,-0.107434,-0.413609,0.300056,0.850056,-0.04601,0.222707,-0.0114921,0.164838,0.415203,-0.294713,0.187536,-0.0324484,0.0921537,-0.23483,-0.152381,0.312211,0.307891,0.011294,-0.0851305,0.300332,-0.0556199,0.342629,0.0971193,0.142128,-0.650731,0.296569,-0.0170683,0.00161196,0.101912,-0.214938,-0.243685,0.0224212,0.266922,-0.00427749,0.000947144,-0.155636,0.373267,0.317644,0.0355001,0.265651,0.276324,-0.0219808,0.0672442,0.200542,0.471807,0.0344986,-0.0202369,-0.303343,0.335097,0.0966534,-0.313922,-0.337812,-0.0620166,-0.185626,0.0933087,0.0755286,-0.0661411,-0.815489,-0.170913,0.235029,-0.180461
education,Bay Area Family Recruiter,-0.0954117,-0.0702975,-0.186445,-0.00935915,0.0431085,0.090021,0.119256,0.0611851,-0.295891,0.231734,-0.170702,0.13069,0.277911,0.222596,-0.302772,0.0669213,0.158227,0.118516,0.0563594,0.154408,0.0242654,0.308068,0.0789735,0.0111396,0.00101944,-0.0425005,0.110333,0.080065,0.255814,-0.0254293,0.36138,0.276812,-0.0964162,0.129942,0.130713,-0.184794,0.0156939,-0.0654218,0.0128636,0.47527,-0.226724,0.314848,-0.0730507,-0.216765,-0.0707427,-0.00603081,0.0699789,-0.14366,-0.364797,-0.0559318,-0.129343,0.197252,-0.107991,-0.11598,0.249636,0.127396,0.0745447,-0.207772,0.0444609,-0.092851,0.116533,-0.00733192,0.208556,-0.187216,0.120188,0.0866311,0.244091,0.146689,0.111703,-0.334957,0.17488,-0.0725898,-0.223959,-0.012875,0.229429,-0.140747,0.0467826,-0.20798,-0.0667032,0.207997,-0.106177,-0.101408,0.0615601,0.151133,-0.254149,0.15218,-0.208574,0.152721,-0.188204,-0.0652586,-0.0742955,-0.019335,0.0973487,0.0386987,0.196887,-0.0808491,-0.013056,-0.107551,-0.0197343,-0.0897458
education,Adult Day Programs/Community Access/Job Coaches,-0.0633724,0.0697162,0.045874,-0.132308,-0.0125692,-0.0697526,0.349191,0.0143475,-0.14405,0.0573575,0.0704745,-0.0583423,0.134514,-0.171774,-0.268729,-0.192195,-0.174414,-0.150645,-0.19068,0.0681866,0.024791,-0.0256565,-0.0828449,-0.116476,0.224361,-0.0730649,-0.283766,-0.201799,0.165288,-0.30559,0.0220558,0.117959,-0.125571,-0.084034,-0.0111401,0.0747539,0.101458,0.0188188,-0.0222562,0.0422242,-0.0116395,0.0998284,-0.00727211,0.000483806,-0.000156656,-

In [11]:
print("Build a basic GBM model")
gbm_model = H2OGradientBoostingEstimator()
gbm_model.train(x = job_title_vecs.names,
                y="category", 
                training_frame = data_split[0], 
                validation_frame = data_split[1])

Build a basic GBM model
gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [12]:
print("Predict!")
print(predict(["school teacher having holidays every month"], w2v_model, gbm_model))
print(predict(["developer with 3+ Java experience, jumping"], w2v_model, gbm_model))
print(predict(["Financial accountant CPA preferred"], w2v_model, gbm_model))

Predict!
Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


predict,accounting,administrative,customerservice,education,foodbeverage,labor
education,0.00183002,0.0057537,0.0109317,0.968453,0.00442612,0.00860562



None
Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


predict,accounting,administrative,customerservice,education,foodbeverage,labor
labor,0.0304531,0.219319,0.260151,0.0279077,0.108545,0.353624



None
Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


predict,accounting,administrative,customerservice,education,foodbeverage,labor
accounting,0.898479,0.0769654,0.0102377,0.00387916,0.00472997,0.00570894



None
